In [1]:
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

import mne
from mne.preprocessing import find_bad_channels_maxwell

/tmp/ipykernel_20436/696113382.py:5: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
SUBJECT_ID = "sub-V1008"
WORKING_DIR = f"/home/s8/Documents/VU/dataset/workdir/{SUBJECT_ID}"
MEG_PATH = f"{WORKING_DIR}/raw_ecg_eog_repaired_meg.fif"

In [3]:
raw = mne.io.read_raw_fif(MEG_PATH, preload=False)

Opening raw data file /home/s8/Documents/VU/dataset/workdir/sub-V1006/raw_ecg_eog_repaired_meg.fif...


FileNotFoundError: fname does not exist: "/home/s8/Documents/VU/dataset/workdir/sub-V1006/raw_ecg_eog_repaired_meg.fif"

In [ ]:
# crop for speedy development, skip when collecting full results
# raw = raw.crop(tmin=0, tmax=360)

In [ ]:
raw.info["bads"] = []
raw_check = raw.copy()
auto_noisy_chs, auto_flat_chs, auto_scores = find_bad_channels_maxwell(
    raw_check,
    return_scores=True,
    verbose=True,
)
print(auto_noisy_chs)  # we should find them!
print(auto_flat_chs)  # none for this dataset

In [ ]:
bads = raw.info["bads"] + auto_noisy_chs + auto_flat_chs
raw.info["bads"] = bads

In [ ]:
# Only select the data for gradiometer channels.
ch_type = "mag"
ch_subset = auto_scores["ch_types"] == ch_type
ch_names = auto_scores["ch_names"][ch_subset]
scores = auto_scores["scores_noisy"][ch_subset]
limits = auto_scores["limits_noisy"][ch_subset]
bins = auto_scores["bins"]  # The the windows that were evaluated.
# We will label each segment by its start and stop time, with up to 3
# digits before and 3 digits after the decimal place (1 ms precision).
bin_labels = [f"{start:3.3f} – {stop:3.3f}" for start, stop in bins]

# We store the data in a Pandas DataFrame. The seaborn heatmap function
# we will call below will then be able to automatically assign the correct
# labels to all axes.
data_to_plot = pd.DataFrame(
    data=scores,
    columns=pd.Index(bin_labels, name="Time (s)"),
    index=pd.Index(ch_names, name="Channel"),
)

# First, plot the "raw" scores.
fig, ax = plt.subplots(1, 2, figsize=(12, 50), layout="constrained")
fig.suptitle(
    f"Automated noisy channel detection: {ch_type}", fontsize=16, fontweight="bold"
)
sns.heatmap(data=data_to_plot, cmap="Reds", cbar_kws=dict(label="Score"), ax=ax[0])
[
    ax[0].axvline(x, ls="dashed", lw=0.25, dashes=(25, 15), color="gray")
    for x in range(1, len(bins))
]
ax[0].set_title("All Scores", fontweight="bold")

# Now, adjust the color range to highlight segments that exceeded the limit.
sns.heatmap(
    data=data_to_plot,
    vmin=np.nanmin(limits),  # bads in input data have NaN limits
    cmap="Reds",
    cbar_kws=dict(label="Score"),
    ax=ax[1],
)
[
    ax[1].axvline(x, ls="dashed", lw=0.25, dashes=(25, 15), color="gray")
    for x in range(1, len(bins))
]
ax[1].set_title("Scores > Limit", fontweight="bold")
plt.show()

In [ ]:
raw_sss = mne.preprocessing.maxwell_filter(
    raw, verbose=True
)

In [ ]:
#raw.pick(["meg"]).plot(duration=2, butterfly=True)
#raw_sss.pick(["meg"]).plot(duration=2, butterfly=True)

In [ ]:
save_path = f"{WORKING_DIR}/raw_sss_repaired_raw_sss.fif"
raw_sss.save(save_path, overwrite=True)